**word-count problem**

In [10]:
# word-count problem

# from collections import Counter

def mapper(text):
  words=text.split()
  keyValue_pair=[(word,1) for word in words]
  return keyValue_pair

def reducer(key, value):
  word_count=sum(value)
  return (key,word_count)

def mapReducer(data, mapper, reducer):
  #map
  inter_result=[]
  for item in data:
    inter_result.extend(mapper(item))


  #reducer
  grouping={}
  for key, value in inter_result:
    if key not in grouping:
      grouping[key]=[]

    grouping[key].append(value)

  result=[]
  for key, value in grouping.items():
    result.append(reducer(key,value))

  return result

text=["Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged."]


res=mapReducer(text,mapper, reducer)
print(res)

print(len(res))

[('Lorem', 2), ('Ipsum', 2), ('is', 1), ('simply', 1), ('dummy', 2), ('text', 2), ('of', 2), ('the', 4), ('printing', 1), ('and', 2), ('typesetting', 1), ('industry.', 1), ('has', 2), ('been', 1), ("industry's", 1), ('standard', 1), ('ever', 1), ('since', 1), ('1500s,', 1), ('when', 1), ('an', 1), ('unknown', 1), ('printer', 1), ('took', 1), ('a', 2), ('galley', 1), ('type', 2), ('scrambled', 1), ('it', 1), ('to', 1), ('make', 1), ('specimen', 1), ('book.', 1), ('It', 1), ('survived', 1), ('not', 1), ('only', 1), ('five', 1), ('centuries,', 1), ('but', 1), ('also', 1), ('leap', 1), ('into', 1), ('electronic', 1), ('typesetting,', 1), ('remaining', 1), ('essentially', 1), ('unchanged.', 1)]
48


**bloom** **fiter**

In [24]:
# bloom fiter
# !pip install mmh3
# !pip install bitarray

import mmh3
from bitarray import bitarray


class BF:
  def __init__(self, size, num_hash):
    self.size=size
    self.num_hash=num_hash
    self.bit_array=bitarray(size)
    self.bit_array.setall(0)

  def hashed(self,item):
    hashes=[]
    for seed in range(self.num_hash):
      hashVal=mmh3.hash(item, seed) % self.size
      hashes.append(hashVal)
    return hashes

  def add(self, item):
    hashes=self.hashed(item)
    for hash in hashes:
      self.bit_array[hash]=1

  def __contains__ (self, item):
    hashes=self.hashed(item)
    for hashVal in hashes:
      if self.bit_array[hashVal]==0:
        return False
    return True


bloom=BF(10000,3)

bloom.add("sumit")
bloom.add("sumit03")
bloom.add("03Sumit")
bloom.add("03sumit")

print("sumit" in bloom)
print("sumit_03" in bloom)





True
False


**aprior algo**

In [27]:
# aprior algo

from collections import defaultdict


def generate_candidates(freq_itemset, k):
  candidates=set()
  for itemset1 in freq_itemset:
    for itemset2 in freq_itemset:
      union_set=itemset1.union(itemset2)
      if len(union_set) == k:
        candidates.add(union_set)
  return candidates

def filter_candidates(dataset, candidates, minSupport):
  itemset_count=defaultdict(int)
  freq_itemset=set()

  for transact in dataset:
    for candidate in candidates:
      if candidate.issubset(transact):
        itemset_count[candidate]+=1

  num_transact =len(dataset)
  for itemset, count in itemset_count.items():
    support = count/num_transact
    if support >= minSupport:
      freq_itemset.add(itemset)

  return freq_itemset

def apriori(dataset, minSupport):
  itemset_count=defaultdict(int)
  freq_itemset=set()

  # count occurance
  for transact in dataset:
    for item in transact:
      itemset_count[frozenset([item])]+=1

  num_transact =len(dataset)
  for item, count in itemset_count.items():
    support = count/num_transact
    if support >= minSupport:
      freq_itemset.add(item)

  k=2
  while freq_itemset:
    candidates=generate_candidates(freq_itemset,k)
    freq_itemset=filter_candidates(dataset, candidates, minSupport)

    for item in freq_itemset:
      print(item)

    k+=1


dataset =[
    {'a','b','c'},
    {'b','c','d'},
    {'a','c','b'},
    {'a','c'},
    {'a','b','c','d'},
    {'b','d'}
]

minSupport=0.5

apriori(dataset,minSupport)

frozenset({'b', 'c'})
frozenset({'b', 'a'})
frozenset({'c', 'a'})
frozenset({'b', 'd'})
frozenset({'b', 'c', 'a'})


**PCY**

In [35]:
import collections
import itertools

def hash(item1, item2):
  """Hash function for the hash table."""
  return (item1 + item2) % 1000

def create_bitmap(hash_table, threshold):
  """Convert the hash table into a bitmap."""
  bit_map = collections.defaultdict(int)
  for key, value in hash_table.items():
    bit_map[value] = 1
  return bit_map

def create_candidate_item_set(dataset):
  """Create a dictionary of all candidate item sets from the data set with their corresponding count."""
  candidate_item_list = collections.defaultdict(int)
  for transaction in dataset:
    for i in range(len(transaction) - 1):
      for j in range(i + 1, len(transaction)):
        item1 = transaction[i]
        item2 = transaction[j]
        hash_value = hash(item1, item2)
        candidate_item_list[hash_value] += 1
  return candidate_item_list

def pcy_algorithm(dataset, min_threshold):
  """Implement the PCY algorithm to find frequent item sets."""
  # Create a hash table of all pairs of items in the data set.
  hash_table = {}
  for transaction in dataset:
    for i in range(len(transaction) - 1):
      for j in range(i + 1, len(transaction)):
        item1 = transaction[i]
        item2 = transaction[j]
        hash_value = hash(item1, item2)
        hash_table[hash_value] = 1 if hash_value not in hash_table else hash_table[hash_value] + 1

  # Convert the hash table into a bitmap.
  bit_map = create_bitmap(hash_table, min_threshold)

  # Create a dictionary of all candidate item sets from the data set with their corresponding count.
  candidate_item_list = create_candidate_item_set(dataset)

  # Find the frequent items from the candidate_item_list.
  frequent_item_set = set()
  for item, count in candidate_item_list.items():
    if count >= min_threshold:
      frequent_item_set.add(item)
  return frequent_item_set

if __name__ == "__main__":
  # Create some self-made data.
  dataset = [[1, 2, 3], [2, 3, 4], [3, 4, 5], [1, 2, 5], [1, 3, 5], [2, 4, 5]]

  # Set the minimum support threshold.
  min_threshold = 2

  # Find the frequent item sets using the PCY algorithm.
  frequent_item_set = pcy_algorithm(dataset, min_threshold)

  # Print the frequent item sets.
  for item in frequent_item_set:
    print(item)


3
4
5
6
7
8
9


### HADOOP

PROCEDURE –
Format the Namenode - Formatting the NameNode is done once when hadoop is installed and not for running hadoop filesystem, else it will delete all the data inside HDFS.
Run this command-

hdfs namenode -format

Now let us start with start the dfs and yarn command, for this this cmd files are available in sbin,we have already given path of sbin in system path variables so we dont need to get inside the sbin folder to run the file. This step involves starting the namenode and datanode with this commands:

start-dfs.cmd

start-yarn.cmd

By perfoming above commands you will get four Apache Distribution windows. Now you can go to this url:http://localhost:9870/ where you can get status of your dfs,namenode and datanode.




Working with HDFS:- Creating a demo file.txt in local file system ,inorder to put it in hdfs usjng hdfs command line tool. Create directory name /user using the following command

hdfs dfs -mkdir /user

hdfs dfs -ls

As now you can see new folder /user is created in hdfs. Now,then we will create a txt file in our local file system ,so that we can put it to hdfs using following commands

hdfs dfs -copyFromLocal address of txt file /user

hdfs dfs -ls/

hdfs dfs -ls/user


As,you can see now RLabInst.txt is now store in our hdfs file system. To see it whats inside the file we can perform this command,

hdfs dfs -cat data.txt

CONCLUSION - We have successfully performed the experiment to implement Hadoop file system.